In [12]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [37]:
# Ackley Function
def objective_function(x):
    n = x.shape[1]  
    sum1 = np.sum(x**2, axis=1) / n
    sum2 = np.sum(np.cos(2 * np.pi * x), axis=1) / n
    return -20 * np.exp(-0.2 * np.sqrt(sum1)) - np.exp(sum2) + 20 + np.e

# # Rosenbrock Function 
# def objective_function(x):
#     return np.sum(100 * (x[:, 1:] - x[:, :-1]**2)**2 + (x[:, :-1] - 1)**2, axis=1)

# # Sphere Function 
# def objective_function(x):
#     return np.sum(x**2, axis=1)

# # Rastrigin Function
# def objective_function(x):
#     n = x.shape[1]  
#     return 10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x), axis=1)

# # Griewank Function 
# def objective_function(x):
#     sum_term = np.sum(x**2, axis=1) / 4000
#     prod_term = np.prod(np.cos(x / np.sqrt(np.arange(1, x.shape[1] + 1))), axis=1)
#     return 1 + sum_term - prod_term

def MFO(nw, dim, iter, xMin, xMax):
    flame_path = []
    moth_path = []
    moths = np.random.uniform(xMin, xMax, (nw, dim))
    best = np.copy(moths)
    best_cost = objective_function(best)

    for it in range(iter):
        n_flame = max(1, int(nw - (nw - 1) * (it / iter)))
        
        sorted_idx = np.argsort(best_cost)
        best = best[sorted_idx]
        best_cost = best_cost[sorted_idx]

        a = -1 - 2 * (it / iter)  
        
        for j in range(nw):
            t = np.random.uniform(-1, 1)
            reference = best[j] if j < n_flame else best[0]  
            distance = np.abs(moths[j] - reference)
            
            scale = 1 - (it / iter)
            moths[j] = reference + scale * distance * np.exp(a * t) * np.cos(2 * np.pi * t) 
            moths[j] = np.clip(moths[j], xMin, xMax)

        flame_path.append(np.copy(best))
        moth_path.append(np.copy(moths))
        moth_costs = objective_function(moths)
        sorted_idx = np.argsort(moth_costs)
        best[:n_flame] = moths[sorted_idx[:n_flame]]
        best_cost[:n_flame] = moth_costs[sorted_idx[:n_flame]]
        # np.random.shuffle(moths)

    best_idx = np.argmin(best_cost)
    print(f"Best solution: {best[best_idx]}, Cost: {best_cost[best_idx]}")
    return best[best_idx], best_cost[best_idx],flame_path,moth_path

nw, dim, iter, xMin, xMax = 70, 2, 50, -10, 10
best_solution, best_cost,moth_path, flame_path = MFO(nw, dim, iter, xMin, xMax)


Best solution: [-5.34234115e-11  1.07229060e-10], Cost: 3.3884761663216523e-10


In [39]:
def create_gif(moth_path, flame_path, xMin, xMax, filename="MFO_movement.gif"):
    frames = []
    x = np.linspace(xMin, xMax, 200)
    y = np.linspace(xMin, xMax, 200)
    X, Y = np.meshgrid(x, y)
    Z = objective_function(np.column_stack([X.ravel(), Y.ravel()])).reshape(X.shape)

    for i, (moths, flames) in enumerate(zip(moth_path, flame_path)):
        fig, ax = plt.subplots(figsize=(10, 8))
        
        contourf = ax.contourf(X, Y, Z, levels=20, cmap='jet')
        
        ax.scatter(flames[:, 0], flames[:, 1], color='orange', edgecolors='black', label='Flames', marker='o')
        ax.scatter(moths[:, 0], moths[:, 1], color='white', edgecolors='black', label='Moths', marker='s')
        ax.scatter(0, 0, c='red', label='Global Min (0,0)', marker='X', s=100)

        ax.set_title(f'Moth-Flame Optimization - Iteration {i+1}')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.legend()
        ax.grid(True)
        plt.colorbar(contourf, ax=ax, label='Value')
        plt.tight_layout()

        fig.canvas.draw()
        img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frames.append(Image.fromarray(img))

        plt.close(fig)  

    if frames:
        frames[0].save(filename, save_all=True, append_images=frames[1:], duration=500, loop=0)
    print(f"GIF saved as {filename}")
    
create_gif(moth_path, flame_path, xMin, xMax)

C:\Users\ali0a\AppData\Local\Temp\ipykernel_9652\2133258273.py:26: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)


GIF saved as MFO_movement.gif
